In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkConf

In [2]:
from pyspark.sql.functions import col, count, desc, lit \
                           ,from_unixtime, hour, array_contains, when

In [3]:
%matplotlib inline
import numpy as np  
import pandas as pd  
import matplotlib.pyplot as plt # plotting

In [4]:
conf=SparkConf()

In [5]:
conf.set("spark.executor.memory", "2g")

In [6]:
spark = SparkSession.builder.config(conf=conf).appName("Lab02").getOrCreate()

In [7]:
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")

# Загружаем данные

In [8]:
train=spark.read.parquet("./data/sna/train")

In [ ]:
print("Количество столбцов:"+str(len(train.columns)))

In [ ]:
print("Количество строк:"+str(train.count()))

 Описание схемы

In [ ]:
train.printSchema()

In [ ]:
train.dtypes

Показать первые 10 строк

In [ ]:
train.show(10,truncate=False)

Подсчитать количество различных значений по столбцам

In [ ]:
for c in train.columns:
   print(c+" distinct "+str(train.select(c).distinct().count())) 

Посчитать количество ненулевых значений

In [ ]:
for c in train.columns:
   print(c+" count not null "+str(train.select(c).filter(col(c).isNotNull()).count()))

Построить топ групп на портале

In [ ]:
topgrp=train.groupBy("metadata_ownerId").agg(count(lit(1)).alias("cnt")).orderBy(col("cnt").desc())

In [ ]:
topgrp.show(50,False)

Построите гистограммы популярности/активности групп на портале по времени суток

In [ ]:
trainGrpByHourP=train.withColumn("hour",hour(from_unixtime(col("audit_timestamp")/1000))) \
               .select("hour").groupBy("hour").agg(count(lit(1)).alias("cnt")).orderBy("hour")

In [ ]:
ptrainGrpByHourP=trainGrpByHourP.toPandas()

In [ ]:
ptrainGrpByHourP.plot("hour","cnt","bar")

In [ ]:
trainGrpByHourA=train.withColumn("hour",hour(from_unixtime(col("metadata_CreatedAt")/1000))) \
               .select("hour").groupBy("hour").agg(count(lit(1)).alias("cnt")).orderBy("hour")

In [ ]:
ptrainGrpByHourA=trainGrpByHourA.toPandas()

In [ ]:
ptrainGrpByHourA.plot("hour","cnt","bar")

Посчитайте корреляцию признаков с целевой переменной 

In [9]:
trainWithTarget=train.withColumn("target",when(array_contains(col("feedback"),"Liked"),1.0) \
                 .otherwise(0.0)).cache()

In [10]:
colsToCheck=list(map(lambda x:x[0],list(filter(lambda x: (x[1]=="bigint") | (x[1]=="int") | (x[1]=="double"), \
            train.dtypes))))

In [11]:
colsToCheck

['instanceId_userId',
 'instanceId_objectId',
 'audit_pos',
 'audit_timestamp',
 'audit_timePassed',
 'audit_resourceType',
 'metadata_ownerId',
 'metadata_createdAt',
 'metadata_authorId',
 'metadata_applicationId',
 'metadata_numCompanions',
 'metadata_numPhotos',
 'metadata_numPolls',
 'metadata_numSymbols',
 'metadata_numTokens',
 'metadata_numVideos',
 'metadata_totalVideoLength',
 'relationsMask',
 'userOwnerCounters_USER_FEED_REMOVE',
 'userOwnerCounters_USER_PROFILE_VIEW',
 'userOwnerCounters_VOTE_POLL',
 'userOwnerCounters_USER_SEND_MESSAGE',
 'userOwnerCounters_USER_DELETE_MESSAGE',
 'userOwnerCounters_USER_INTERNAL_LIKE',
 'userOwnerCounters_USER_INTERNAL_UNLIKE',
 'userOwnerCounters_USER_STATUS_COMMENT_CREATE',
 'userOwnerCounters_PHOTO_COMMENT_CREATE',
 'userOwnerCounters_MOVIE_COMMENT_CREATE',
 'userOwnerCounters_USER_PHOTO_ALBUM_COMMENT_CREATE',
 'userOwnerCounters_COMMENT_INTERNAL_LIKE',
 'userOwnerCounters_USER_FORUM_MESSAGE_CREATE',
 'userOwnerCounters_PHOTO_MARK_CREA

In [12]:
trainWithTarget.corr("instanceId_userId","target")

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:61149)
Traceback (most recent call last):
  File "/Users/alexeykvashnin/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-12-1b9180b44479>", line 1, in <module>
    trainWithTarget.corr("instanceId_userId","target")
  File "/Users/alexeykvashnin/opt/anaconda3/lib/python3.7/site-packages/pyspark/sql/dataframe.py", line 1908, in corr
    return self._jdf.stat().corr(col1, col2, method)
  File "/Users/alexeykvashnin/opt/anaconda3/lib/python3.7/site-packages/py4j/java_gateway.py", line 1257, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/Users/alexeykvashnin/opt/anaconda3/lib/python3.7/site-packages/pyspark/sql/utils.py", line 63, in deco
    return f(*a, **kw)
  File "/Users/alexeykvashnin/opt/anaconda3/lib/python3.7/site-packages/py4j/

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:61149)
Traceback (most recent call last):
  File "/Users/alexeykvashnin/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-12-1b9180b44479>", line 1, in <module>
    trainWithTarget.corr("instanceId_userId","target")
  File "/Users/alexeykvashnin/opt/anaconda3/lib/python3.7/site-packages/pyspark/sql/dataframe.py", line 1908, in corr
    return self._jdf.stat().corr(col1, col2, method)
  File "/Users/alexeykvashnin/opt/anaconda3/lib/python3.7/site-packages/py4j/java_gateway.py", line 1257, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/Users/alexeykvashnin/opt/anaconda3/lib/python3.7/site-packages/pyspark/sql/utils.py", line 63, in deco
    return f(*a, **kw)
  File "/Users/alexeykvashnin/opt/anaconda3/lib/python3.7/site-packages/py4j/

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:61149)
Traceback (most recent call last):
  File "/Users/alexeykvashnin/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-12-1b9180b44479>", line 1, in <module>
    trainWithTarget.corr("instanceId_userId","target")
  File "/Users/alexeykvashnin/opt/anaconda3/lib/python3.7/site-packages/pyspark/sql/dataframe.py", line 1908, in corr
    return self._jdf.stat().corr(col1, col2, method)
  File "/Users/alexeykvashnin/opt/anaconda3/lib/python3.7/site-packages/py4j/java_gateway.py", line 1257, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/Users/alexeykvashnin/opt/anaconda3/lib/python3.7/site-packages/pyspark/sql/utils.py", line 63, in deco
    return f(*a, **kw)
  File "/Users/alexeykvashnin/opt/anaconda3/lib/python3.7/site-packages/py4j/

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:61149)
Traceback (most recent call last):
  File "/Users/alexeykvashnin/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-12-1b9180b44479>", line 1, in <module>
    trainWithTarget.corr("instanceId_userId","target")
  File "/Users/alexeykvashnin/opt/anaconda3/lib/python3.7/site-packages/pyspark/sql/dataframe.py", line 1908, in corr
    return self._jdf.stat().corr(col1, col2, method)
  File "/Users/alexeykvashnin/opt/anaconda3/lib/python3.7/site-packages/py4j/java_gateway.py", line 1257, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/Users/alexeykvashnin/opt/anaconda3/lib/python3.7/site-packages/pyspark/sql/utils.py", line 63, in deco
    return f(*a, **kw)
  File "/Users/alexeykvashnin/opt/anaconda3/lib/python3.7/site-packages/py4j/

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:61149)
Traceback (most recent call last):
  File "/Users/alexeykvashnin/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-12-1b9180b44479>", line 1, in <module>
    trainWithTarget.corr("instanceId_userId","target")
  File "/Users/alexeykvashnin/opt/anaconda3/lib/python3.7/site-packages/pyspark/sql/dataframe.py", line 1908, in corr
    return self._jdf.stat().corr(col1, col2, method)
  File "/Users/alexeykvashnin/opt/anaconda3/lib/python3.7/site-packages/py4j/java_gateway.py", line 1257, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/Users/alexeykvashnin/opt/anaconda3/lib/python3.7/site-packages/pyspark/sql/utils.py", line 63, in deco
    return f(*a, **kw)
  File "/Users/alexeykvashnin/opt/anaconda3/lib/python3.7/site-packages/py4j/

...

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:61149)
Traceback (most recent call last):
  File "/Users/alexeykvashnin/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-12-1b9180b44479>", line 1, in <module>
    trainWithTarget.corr("instanceId_userId","target")
  File "/Users/alexeykvashnin/opt/anaconda3/lib/python3.7/site-packages/pyspark/sql/dataframe.py", line 1908, in corr
    return self._jdf.stat().corr(col1, col2, method)
  File "/Users/alexeykvashnin/opt/anaconda3/lib/python3.7/site-packages/py4j/java_gateway.py", line 1257, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/Users/alexeykvashnin/opt/anaconda3/lib/python3.7/site-packages/pyspark/sql/utils.py", line 63, in deco
    return f(*a, **kw)
  File "/Users/alexeykvashnin/opt/anaconda3/lib/python3.7/site-packages/py4j/

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:61149)
Traceback (most recent call last):
  File "/Users/alexeykvashnin/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-12-1b9180b44479>", line 1, in <module>
    trainWithTarget.corr("instanceId_userId","target")
  File "/Users/alexeykvashnin/opt/anaconda3/lib/python3.7/site-packages/pyspark/sql/dataframe.py", line 1908, in corr
    return self._jdf.stat().corr(col1, col2, method)
  File "/Users/alexeykvashnin/opt/anaconda3/lib/python3.7/site-packages/py4j/java_gateway.py", line 1257, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/Users/alexeykvashnin/opt/anaconda3/lib/python3.7/site-packages/pyspark/sql/utils.py", line 63, in deco
    return f(*a, **kw)
  File "/Users/alexeykvashnin/opt/anaconda3/lib/python3.7/site-packages/py4j/

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:61149)
Traceback (most recent call last):
  File "/Users/alexeykvashnin/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-12-1b9180b44479>", line 1, in <module>
    trainWithTarget.corr("instanceId_userId","target")
  File "/Users/alexeykvashnin/opt/anaconda3/lib/python3.7/site-packages/pyspark/sql/dataframe.py", line 1908, in corr
    return self._jdf.stat().corr(col1, col2, method)
  File "/Users/alexeykvashnin/opt/anaconda3/lib/python3.7/site-packages/py4j/java_gateway.py", line 1257, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/Users/alexeykvashnin/opt/anaconda3/lib/python3.7/site-packages/pyspark/sql/utils.py", line 63, in deco
    return f(*a, **kw)
  File "/Users/alexeykvashnin/opt/anaconda3/lib/python3.7/site-packages/py4j/

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:61149)
Traceback (most recent call last):
  File "/Users/alexeykvashnin/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-12-1b9180b44479>", line 1, in <module>
    trainWithTarget.corr("instanceId_userId","target")
  File "/Users/alexeykvashnin/opt/anaconda3/lib/python3.7/site-packages/pyspark/sql/dataframe.py", line 1908, in corr
    return self._jdf.stat().corr(col1, col2, method)
  File "/Users/alexeykvashnin/opt/anaconda3/lib/python3.7/site-packages/py4j/java_gateway.py", line 1257, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/Users/alexeykvashnin/opt/anaconda3/lib/python3.7/site-packages/pyspark/sql/utils.py", line 63, in deco
    return f(*a, **kw)
  File "/Users/alexeykvashnin/opt/anaconda3/lib/python3.7/site-packages/py4j/

Py4JNetworkError: An error occurred while trying to connect to the Java server (127.0.0.1:61149)

In [ ]:
for c in colsToCheck:
  print(trainWithTarget.corr(c,"target"))

In [ ]:
spark.stop